In [ ]:
from nwb_project_analytics.gitstats import NWBGitInfo, GitRepos, GitRepo
from nwb_project_analytics.renderstats import RenderCommitStats

In [ ]:
import os
from github import Github, Label
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime
from collections import defaultdict
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# Configure inputs

In [ ]:
# save results to
output_dir = os.path.join(os.getcwd(), 'plots/')  
# save figures
save_figs = True
# exclude issues raised by core devs from response-time analysis
DEV_USERNAMES = NWBGitInfo.CORE_DEVELOPERS 
# which repos to use. 
# Set to NWBGitInfo.CORE_API_REPOS to use only main API NWB repos. 
# Set to NWBGitInfo.GIT_REPOS to use all main NWB 2 repos
REPOS = GitRepos([(k, NWBGitInfo.GIT_REPOS[k]) for k in ["PyNWB", 
                                                         "HDMF", 
                                                         "MatNWB", 
                                                         "NWB_Schema",
                                                         "NWBInspector",
                                                         "NWBWidgets",
                                                         "HDMF_Zarr",
                                                         "HDMF_Common_Schema",
                                                         "HDMF_DocUtils"]])
# Set datetime to filter issues older than START. E.g., set to  datetime(2021, 5, 1)
START = datetime(2022, 1, 1) # NWBGitInfo.NWB1_DEPRECATION_DATE
start_str = START.strftime("%Y-%m-%d")
end_str = datetime.now().strftime("%Y-%m-%d")

We need an API key from GitHub to access the API. See https://docs.github.com/en/github/authenticating-to-github/keeping-your-account-and-data-secure/creating-a-personal-access-token how to generate an access token.

**WARNING:** Never check in a token to the repo
**WARNING:** Only use tokens with read only access (never write access) to avoid accidental changes to the repo

In [ ]:
with open('ghk.txt', 'r') as f:
    API_KEY = f.read().rstrip("\n")
g = Github(API_KEY)

# Compile commit statistics as dataframe

In [ ]:
commits_dfs = {repo: repo.get_commits_as_dataframe(since=START, github_obj=g, tqdm=tqdm)
               for repo in tqdm(REPOS.values(), position=0, desc='repos')}

In [ ]:
cumulative = False
for repo, cdf in commits_dfs.items():
    if cumulative:
        fig, ax = RenderCommitStats.plot_commit_cumulative_additions_and_deletions(
            commits=cdf,
            repo_name=repo.repo)
    else:
        fig, ax = RenderCommitStats.plot_commit_additions_and_deletions(
            commits=cdf,
            repo_name=repo.repo,
            xaxis_dates=False,
            bar_width=0.9,
            color_additions=RenderCommitStats.COLOR_ADDITIONS,
            color_deletions=RenderCommitStats.COLOR_DELETIONS,
            xticks_rotate=90
        )
    if save_figs:
        plt.savefig(os.path.join(output_dir, '%s_commit_add_delete_stats_%s_%s.pdf' % (repo.repo, start_str, end_str)))
        plt.savefig(os.path.join(output_dir, '%s_commit_add_delete_stats_%s_%s.png' % (repo.repo, start_str, end_str)))
    plt.show()

In [ ]:
fig, ax =  RenderCommitStats.plot_commit_additions_and_deletions_summary(
    commits=commits_dfs,
    bar_width=0.8,
    color_additions=RenderCommitStats.COLOR_ADDITIONS,
    color_deletions=RenderCommitStats.COLOR_DELETIONS,
    xticks_rotate= 45,
    start_date = START,
    end_date = datetime.now()
)
plt.tight_layout()
if save_figs:
    plt.savefig(os.path.join(output_dir, 'NWB_ALL_commit_add_delete_stats_%s_%s.pdf' % (start_str, end_str)))
    plt.savefig(os.path.join(output_dir, 'NWB_ALL_commit_add_delete_stats_%s_%s.png' % (start_str, end_str)))
plt.show()